$$\textbf{\Huge Technical Assignment W3}$$

$$\textbf{\Huge Part1}$$

In [1]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
!unzip imdb-dataset-of-50k-movie-reviews.zip

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 62%|████████████████████████▎              | 16.0M/25.7M [00:00<00:00, 166MB/s]
100%|███████████████████████████████████████| 25.7M/25.7M [00:00<00:00, 211MB/s]
Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


$$Step 1$$

In [1]:
import pandas as pd

file_path = '/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv'
#file_path = 'IMDB Dataset.csv'

# Load the IMDB dataset
imdb_data = pd.read_csv(file_path)
#imdb_data = imdb_data.sample(n=2000, random_state=42)
# Display the first few rows of the dataset to verify
imdb_data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


$$Step 2$$

In [2]:
imdb_data['label'] = imdb_data['sentiment'].map({'positive': 1, 'negative': 0})

# Retain only the 'review' and 'sentiment' columns
imdb_data = imdb_data[['review', 'label']]



In [4]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    imdb_data["review"], imdb_data["label"], test_size=0.2, random_state=42
)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42
)

$$Step 3$$

In [5]:
!pip install transformers datasets torch


In [6]:

import torch
from datasets import Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments


In [7]:
train_dataset = Dataset.from_dict({"text": train_texts.tolist(), "label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"text": val_texts.tolist(), "label": val_labels.tolist()})
test_dataset = Dataset.from_dict({"text": test_texts.tolist(), "label": test_labels.tolist()})

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
train_dataset = train_dataset.remove_columns(["text"])
val_dataset = val_dataset.remove_columns(["text"])
test_dataset = test_dataset.remove_columns(["text"])


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/32000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [8]:
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


$$Step 4$$

In [9]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os
os.environ["WANDB_DISABLED"] = "true"

model=DistilBertForSequenceClassification.from_pretrained(
        "distilbert-base-uncased",
        num_labels=2,
    )


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=1)
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}


training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="no",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,  
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="epoch",
    fp16=True,
    disable_tqdm=True,
    report_to="none",  # disable wandb
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train and Evaluate
trainer.train()
metrics = trainer.evaluate() 
print("Validation Metrics:", metrics)




model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-9-d3b0e8dbfd60>:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'loss': 0.2729, 'grad_norm': 267290.3125, 'learning_rate': 2.5e-05, 'epoch': 1.0}
{'eval_loss': 0.22469815611839294, 'eval_accuracy': 0.9115, 'eval_precision': 0.8912890349847382, 'eval_recall': 0.9393714427122, 'eval_f1': 0.9146987951807228, 'eval_runtime': 36.3515, 'eval_samples_per_second': 220.074, 'eval_steps_per_second': 6.877, 'epoch': 1.0}


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'loss': 0.1278, 'grad_norm': 106537.2421875, 'learning_rate': 0.0, 'epoch': 2.0}
{'eval_loss': 0.27358222007751465, 'eval_accuracy': 0.918875, 'eval_precision': 0.9223107569721115, 'eval_recall': 0.9166048007918832, 'eval_f1': 0.9194489263994042, 'eval_runtime': 36.3397, 'eval_samples_per_second': 220.145, 'eval_steps_per_second': 6.88, 'epoch': 2.0}
{'train_runtime': 975.6825, 'train_samples_per_second': 65.595, 'train_steps_per_second': 2.05, 'train_loss': 0.20033306121826172, 'epoch': 2.0}


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.27358222007751465, 'eval_accuracy': 0.918875, 'eval_precision': 0.9223107569721115, 'eval_recall': 0.9166048007918832, 'eval_f1': 0.9194489263994042, 'eval_runtime': 36.388, 'eval_samples_per_second': 219.853, 'eval_steps_per_second': 6.87, 'epoch': 2.0}
Validation Metrics: {'eval_loss': 0.27358222007751465, 'eval_accuracy': 0.918875, 'eval_precision': 0.9223107569721115, 'eval_recall': 0.9166048007918832, 'eval_f1': 0.9194489263994042, 'eval_runtime': 36.388, 'eval_samples_per_second': 219.853, 'eval_steps_per_second': 6.87, 'epoch': 2.0}


In [10]:
# Final evaluation on test set
test_metrics = trainer.evaluate(test_dataset)   
print("Test Set Metrics:", test_metrics)

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.27002236247062683, 'eval_accuracy': 0.9167, 'eval_precision': 0.9152843601895735, 'eval_recall': 0.9198253621750347, 'eval_f1': 0.9175492427991685, 'eval_runtime': 46.7879, 'eval_samples_per_second': 213.731, 'eval_steps_per_second': 6.69, 'epoch': 2.0}
Test Set Metrics: {'eval_loss': 0.27002236247062683, 'eval_accuracy': 0.9167, 'eval_precision': 0.9152843601895735, 'eval_recall': 0.9198253621750347, 'eval_f1': 0.9175492427991685, 'eval_runtime': 46.7879, 'eval_samples_per_second': 213.731, 'eval_steps_per_second': 6.69, 'epoch': 2.0}


$$Step 5$$

In [11]:
model.save_pretrained("fine_tuned_DistilBert_Xu")
tokenizer.save_pretrained("fine_tuned_DistilBert_Xu")


('fine_tuned_DistilBert_Xu/tokenizer_config.json',
 'fine_tuned_DistilBert_Xu/special_tokens_map.json',
 'fine_tuned_DistilBert_Xu/vocab.txt',
 'fine_tuned_DistilBert_Xu/added_tokens.json')

In [12]:
from huggingface_hub import notebook_login

notebook_login()

In [13]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from huggingface_hub import HfApi

# Define the repo name 
repo_name = "Wenfi/fine-tuned-DistilBert_Xu"

# Push model to Hugging Face Hub
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Wenfi/fine-tuned-DistilBert_Xu/commit/484139a6642cf7c0b8746f2eacb416a0a11ef73b', commit_message='Upload tokenizer', commit_description='', oid='484139a6642cf7c0b8746f2eacb416a0a11ef73b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Wenfi/fine-tuned-DistilBert_Xu', endpoint='https://huggingface.co', repo_type='model', repo_id='Wenfi/fine-tuned-DistilBert_Xu'), pr_revision=None, pr_num=None)

Verify the model 

In [14]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_loaded = AutoModelForSequenceClassification.from_pretrained("Wenfi/fine-tuned-DistilBert_Xu")
tokenizer_loaded = AutoTokenizer.from_pretrained("Wenfi/fine-tuned-DistilBert_Xu")

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [15]:
inputs = tokenizer_loaded("This is a test sentence.", return_tensors="pt")
outputs = model_loaded(**inputs)
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.1587,  0.4488]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


 Link to the fine tuned model: https://huggingface.co/Wenfi/fine-tuned-DistilBert_Xu

[Click here to view my model on Hugging Face](https://huggingface.co/Wenfi/fine-tuned-DistilBert_Xu)

$$\textbf{\Huge Part 2}$$

$$Step6-7:app.py$$

Step 8:  Curl:
 curl -X POST "http://127.0.0.1:8000/analyze/" ^
     -H "Content-Type: application/json" ^
     -d "{ \"text\": \"I love this product, it's amazing!\", \"model\": \"llama\" }"

curl -X POST "http://127.0.0.1:8000/analyze/" ^
     -H "Content-Type: application/json" ^
     -d "{ \"text\": \"I love this product, it's amazing!\", \"model\": \"custom\" }"

     Python: test_app.py


Step 9: Clear and reusable prompt for the Llama 3 model in Groq Cloud could be:

Determine the sentiment of this text as positive, negative, or neutral, and provide the confidence level for each sentiment, Please make sure to explain the reasoning behind the sentiment classification.: {text}


Step 10: 

UI address:http//localhost:5173/
If I type: This smovie is boring,  model: Llama 3, 
result: Sentiment: the sentiment of this text is **negative**. the word "boring" has a negative connotation, indicating that the speaker did not enjoy the movie.UI address: 
Confidence: 0.9

If I choose model: Custom Model
Result: Sentiment: LABEL_0
Confidence: 0.9976978898048401


Step 11-12: https://github.com/wenxu-fi/sentiment-analysis/

Step 13: 